In [1]:
import json
import random
import numpy as np
import datetime
import webbrowser
import nltk
import tkinter as tk
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout
from keras.optimizers.legacy import SGD

In [2]:
# Download NLTK data if not already installed
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmed\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Load intents from intents.json
with open('intents.json') as file:
    data = json.load(file)

In [4]:
lemmatizer = nltk.stem.WordNetLemmatizer()
intents = data['intents']

In [5]:
# Initialize chatbot data
words = []
classes = []
documents = []
responses = []

for intent in intents:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
    responses.append(intent['responses'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w.isalpha()]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

In [6]:
# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(
        word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = list(training)

X = np.array([i[0] for i in training])
y = np.array([i[1] for i in training])

In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(X[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y[0]), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               12160     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 21)                1365      
                                                                 
Total params: 21781 (85.08 KB)
Trainable params: 21781 (85.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
hist = model.fit(np.array(X), np.array(y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
17/17 [==============================] - 1s 3ms/step - loss: 3.0684 - accuracy: 0.0488
Epoch 2/200
17/17 [==============================] - 0s 2ms/step - loss: 2.9937 - accuracy: 0.1463
Epoch 3/200
17/17 [==============================] - 0s 2ms/step - loss: 2.9868 - accuracy: 0.1098
Epoch 4/200
17/17 [==============================] - 0s 2ms/step - loss: 2.9366 - accuracy: 0.1463
Epoch 5/200
17/17 [==============================] - 0s 2ms/step - loss: 2.8536 - accuracy: 0.1707
Epoch 6/200
17/17 [==============================] - 0s 2ms/step - loss: 2.7937 - accuracy: 0.2195
Epoch 7/200
17/17 [==============================] - 0s 2ms/step - loss: 2.7674 - accuracy: 0.2073
Epoch 8/200
17/17 [==============================] - 0s 2ms/step - loss: 2.6662 - accuracy: 0.2805
Epoch 9/200
17/17 [==============================] - 0s 2ms/step - loss: 2.6870 - accuracy: 0.2317
Epoch 10/200
17/17 [==============================] - 0s 2ms/step - loss: 2.5289 - accuracy: 0.2683
Epoch 11/

17/17 [==============================] - 0s 2ms/step - loss: 0.2283 - accuracy: 0.9512
Epoch 84/200
17/17 [==============================] - 0s 2ms/step - loss: 0.3181 - accuracy: 0.9512
Epoch 85/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2704 - accuracy: 0.9756
Epoch 86/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2430 - accuracy: 0.9634
Epoch 87/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1075 - accuracy: 0.9878
Epoch 88/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1687 - accuracy: 0.9634
Epoch 89/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1890 - accuracy: 0.9512
Epoch 90/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2052 - accuracy: 0.9390
Epoch 91/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1357 - accuracy: 0.9634
Epoch 92/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2275 - accuracy: 0.9512
Epoch 93/200


17/17 [==============================] - 0s 2ms/step - loss: 0.0414 - accuracy: 1.0000
Epoch 165/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0921 - accuracy: 0.9756
Epoch 166/200
17/17 [==============================] - 0s 2ms/step - loss: 0.2080 - accuracy: 0.9512
Epoch 167/200
17/17 [==============================] - 0s 3ms/step - loss: 0.0826 - accuracy: 0.9878
Epoch 168/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0937 - accuracy: 0.9756
Epoch 169/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1117 - accuracy: 0.9634
Epoch 170/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0827 - accuracy: 0.9756
Epoch 171/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0919 - accuracy: 0.9756
Epoch 172/200
17/17 [==============================] - 0s 2ms/step - loss: 0.0635 - accuracy: 1.0000
Epoch 173/200
17/17 [==============================] - 0s 2ms/step - loss: 0.1054 - accuracy: 0.9634
Epoc

In [9]:
# Save trained model
model.save('chatbot_model.h5', hist)

# Load saved model
model = load_model('chatbot_model.h5')

C:\Users\ahmed\anaconda3\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Return a bag of words: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(
        word.lower()) for word in sentence_words]
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
    return np.array(bag)

In [11]:
# Main chatbot interaction
print("Hi! I'm Chat-Bot, How can I assist you today?\n" + '^'*45)

while True:
    user_input = input("\nYou: ")
    
    p = bow(user_input, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return_list.sort(key=lambda x: x['probability'], reverse=True)
    
    intent = return_list[0]['intent']

    for i in intents:
        if i['tag'] == intent:
            print("Bot:", random.choice(i['responses']))

    if intent == 'exit':
        break
    elif intent == 'weather':
        webbrowser.open("https://openweathermap.org")
    elif intent == 'news':
        webbrowser.open("https://news.google.com")
    elif intent == 'time':
        print("Bot:", datetime.datetime.now().strftime("%d/%B/%Y %I:%M:%S %p"))
    elif intent == 'search':
        search_query = input("You: ")
        print(f"Bot: Searching on Google for: {search_query}")
        webbrowser.open(f"https://www.google.com/search?q={search_query}")

Hi! I'm Chat-Bot, How can I assist you today?
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

You: Hi
1/1 [==============================] - 0s 89ms/step
Bot: Good to see you! What can I do for you?

You: How are you?
1/1 [==============================] - 0s 21ms/step
Bot: All good! How can I assist you today?

You: What can you do?
1/1 [==============================] - 0s 23ms/step
Bot: You can ask me about the weather, news, time, jokes, or any questions you have. Just type your query, and I'll do my best to assist.

You: Who developed you?
1/1 [==============================] - 0s 22ms/step
Bot: I was created by Ahmed Nasser.

You: Who is Ahmed Nasser?
1/1 [==============================] - 0s 29ms/step
Bot: You can learn more about Ahmed Nasser by visiting his LinkedIn profile: https://linkedin.com/in/ahmednasser1601, also find his projects on GitHub: https://github.com/AhmedNasser1601.

You: Make me laugh
1/1 [==============================] - 0s 22ms/step
Bot: I ate a clock yest